In [ ]:
import os, time, datetime, re, copy
import pandas as pd
import numpy as np
from selenium import webdriver
from itertools import groupby

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {
    "profile.default_content_settings.popups": 0,
    "download.default_directory": "./",
    "directory_upgrade": True,
}
options.add_experimental_option("prefs", prefs)
browser = webdriver.Chrome(options=options)

split_url = "http://www.moneycontrol.com/stocks/marketinfo/splits/index.php?sel_year="
bonus_url = "http://www.moneycontrol.com/stocks/marketinfo/bonus/index.php?sel_year="
url = [split_url, bonus_url]

def getBseCorpAct(browser):
    browser.get("https://www.bseindia.com/corporates/corporate_act.aspx")

    browser.find_element_by_name("ctl00$ContentPlaceHolder1$txtDate").click()
    browser.execute_script("javascript:setCalendarControlDate(" + str(years[0]) + ",1,1)")

    browser.find_element_by_name("ctl00$ContentPlaceHolder1$txtTodate").click()
    browser.execute_script("javascript:setCalendarControlDate(" + str(yearT) + "," + str(monthT).lstrip("0") + "," + str(dayT).lstrip("0") + ")")

    browser.find_element_by_name("ctl00$ContentPlaceHolder1$btnSubmit").click()
    # directly downloads to preferref location set in driver preferences
    browser.execute_script("javascript:__doPostBack('ctl00$ContentPlaceHolder1$lnkDownload','')")
    while "Corporate_Actions.csv" not in os.listdir():
        time.sleep(1)
    # data = pd.read_csv(directory + "Corporate_Actions.csv", index_col = None)
    data = pd.read_csv("./Corporate_Actions.csv")
    data.columns = [x.lower().replace(" ", "_").replace("\t", "") for x in data.columns]
    data["purpose"] = data.purpose.apply(lambda x: x.lower())
    data = data[[True if "split" in x or "bonus" in x else False for x in data["purpose"]]]
    data.reset_index(drop=True, inplace=True)
    # data.to_csv(directory + "Corporate_Actions.csv", index = None)
    data.to_csv("./Corporate_Actions.csv", index=None)


def getIDentifiers(link, browser):
    print(len([x for x in link.split("/") if x != ""]))
    if len([x for x in link.split("/") if x != ""]) != 7:
        return pd.DataFrame([[np.nan] * 4], columns=["bse", "nse", "isin", "sector"])
    browser.get(link)
    print(link)
    time.sleep(2)
    idList = browser.find_element_by_xpath('//div[@class = "FL gry10"]').text.split("|")[:-1]
    idList = [x.strip().split(":") for x in idList]
    return pd.DataFrame.from_dict({val[0].strip().lower(): [val[1].strip()] for val in idList})


def getCorpActData(url, years, act):
    i = 0
    mssg = "Getting Data for Splits"
    if act == "Bonus":
        i = 1
        mssg = "Getting Data for Bonus"
    final_df = None
    for year in years:
        print(mssg)
        link = url[i] + str(year)
        browser.get(link)
        time.sleep(3)
        content = browser.find_element_by_xpath('//div[@class="MT15"]/table[@class="b_12 dvdtbl tbldata14"]').get_attribute("outerHTML")
        links = browser.find_elements_by_xpath('//div[@class="MT15"]/table[@class="b_12 dvdtbl tbldata14"]//td[@class="dvd_brdb"]/a')
        links = [x.get_attribute("href") for x in links]
        links = links
        df = pd.read_html(content)[0]
        if i == 0:
            df.drop([0], inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.columns = ["company", "old", "new", "split_date"]
            idListDf = [getIDentifiers(x, browser) for x in links]
            idDf = pd.concat(idListDf, axis=0)
            idDf.reset_index(drop=True, inplace=True)
            df = pd.concat([df, idDf], axis=1)
        else:
            df.drop([0, 1], inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.columns = ["company", "bonus_ratio", "ex_bonus_date", "announcement_date", "record_date"]
            bonusDf = pd.DataFrame(df["bonus_ratio"].apply(lambda x: x.split(":")).tolist(), columns=["old", "new"])
            df = pd.concat([df, bonusDf], axis=1)
            idListDf = [getIDentifiers(x, browser) for x in links]
            idDf = pd.concat(idListDf, axis=0)
            idDf.reset_index(drop=True, inplace=True)
            df = pd.concat([df, idDf], axis=1)
        df["company"] = df["company"].apply(lambda x: str(x).replace("  Add to Watchlist  Add to Portfolio", ""))
        final_df = pd.concat([final_df, df], axis=0)
    return final_df
getBseCorpAct(browser)

In [ ]:
def adjustmentCalculator(data, typeOf, old, new):
    if typeOf == "split":
        data["rate"] = data["rate"] * (new / old)
        data["qty"] = data["qty"] * (old / new)
    elif typeOf == "bonus":  # will include other as well soon like reverse split
        cols = list(data.columns)
        data["remainder"] = [x % new for x in data["qty"]]
        data["quotient"] = [int(x / new) for x in data["qty"]]
        data["rate"] = data["qty"] * data["rate"] / (data["qty"] + data["quotient"])
        data["qty"] = data["qty"] + data["quotient"]
        data = data[cols]
    return data

In [ ]:
# Loading the datasets
splitDF = pd.read_csv("./split.csv")
splitDF["split_date"] = pd.to_datetime(splitDF["split_date"], format="%d-%m-%Y").dt.date
# Keep only records where we have nse symbol, getting bse symbol from bse website
splitDF.dropna(axis=0, subset=["nse"], inplace=True)
splitDF.reset_index(drop=True, inplace=True)
splitDF = splitDF[["nse", "split_date", "old", "new"]]
splitDF.columns = ["symbol", "date", "old", "new"]
splitDF["type"] = "split"


# bonusDF = pd.read_csv(directory + "bonus.csv", index_col = None)
bonusDF = pd.read_csv("./bonus.csv")
bonusDF["split_date"] = pd.to_datetime(bonusDF["record_date"], format="%d-%m-%Y").dt.date
# Keep only records where we have nse symbol, getting bse symbol from bse website
bonusDF.dropna(axis=0, subset=["nse"], inplace=True)
bonusDF.reset_index(drop=True, inplace=True)
bonusDF = bonusDF[["nse", "split_date", "old", "new"]]
bonusDF.columns = ["symbol", "date", "old", "new"]
bonusDF["type"] = "bonus"


# bseCorpDF = pd.read_csv(directory + "Corporate_Actions.csv", index_col = None)
bseCorpDF = pd.read_csv("./Corporate_Actions.csv")
bseCorpDF["ex_date_form"] = pd.to_datetime(bseCorpDF["ex_date"], format="%d %b %Y").dt.date

# Creating split and bonuses
bseCorpSplitDf = bseCorpDF[["split" in x for x in bseCorpDF.purpose]]
bseCorpSplitDf.reset_index(drop=True, inplace=True)
bseCorpSplitDf = bseCorpSplitDf[["security_name", "ex_date_form", "purpose"]]
old_new = pd.DataFrame(bseCorpSplitDf.purpose.apply(lambda x: re.findall(r"\d+", x)[-2:]).tolist(), columns=["old", "new"])
bseCorpSplitDf = pd.concat([bseCorpSplitDf, old_new], axis=1)
bseCorpSplitDf = bseCorpSplitDf[["security_name", "ex_date_form", "old", "new"]]
bseCorpSplitDf.columns = ["symbol", "date", "old", "new"]
bseCorpSplitDf["type"] = "split"

bseCorpBonusDf = bseCorpDF[["bonus" in x for x in bseCorpDF.purpose]]
bseCorpBonusDf.reset_index(drop=True, inplace=True)
bseCorpBonusDf = bseCorpBonusDf[["security_name", "ex_date_form", "purpose"]]
old_new = pd.DataFrame(bseCorpBonusDf.purpose.apply(lambda x: re.findall(r"\d+", x)[:2]).tolist(), columns=["old", "new"])
bseCorpBonusDf = pd.concat([bseCorpBonusDf, old_new], axis=1)
bseCorpBonusDf = bseCorpBonusDf[["security_name", "ex_date_form", "old", "new"]]
bseCorpBonusDf.columns = ["symbol", "date", "old", "new"]
bseCorpBonusDf["type"] = "bonus"


corpAcDF = pd.concat([splitDF, bseCorpSplitDf, bonusDF, bseCorpBonusDf], axis=0, ignore_index=True)
corpAcDF["symbol"] = [x.strip() for x in corpAcDF["symbol"]]
corpAcDF["old"] = [int(x) for x in corpAcDF["old"]]
corpAcDF["new"] = [int(x) for x in corpAcDF["new"]]
corpAcDF.drop_duplicates(inplace=True)
corpAcDF.reset_index(inplace=True, drop=True)

# Getting unique symbols in my list and adjusting them for corporate action
symbols = np.unique(tradeDF.symbol)
symbolGroup = tradeDF.groupby(["symbol"])
symbolsCorpAcGroup = corpAcDF.groupby(["symbol"])

symbolGroupDict = {}
for symbol in symbols:
    print(symbol)
    # Tradebook
    df = symbolGroup.get_group(symbol)
    df.sort_values("date", inplace=True)
    df.reset_index(inplace=True, drop=True)
    if len(np.unique(df["type"])) > 1:
        # print(symbol,"Length Greater")
        if any([symbol in x for x in corpAcDF["symbol"]]):
            try:
                dfCorp = symbolsCorpAcGroup.get_group(symbol)
            except KeyError as e:
                symbolGroupDict[symbol] = pnlCalculator(data=df)
                continue
            # corporate Action
            dfCorp.sort_values("date", inplace=True)
            dfCorp.reset_index(inplace=True, drop=True)
            for _, row in dfCorp.iterrows():
                try:
                    tillIndex = np.min(np.where(df.date == row["date"]))
                except ValueError as e:
                    continue
                tradeDF = df.iloc[:tillIndex].reset_index(drop=True)
                tradeDF = adjustmentCalculator(tradeDF, row["type"], row["old"], row["new"])
                df.iloc[:tillIndex] = tradeDF

        symbolGroupDict[symbol] = pnlCalculator(data=df)
    else:
        # print(symbol,"Length Lower")
        symbolGroupDict[symbol] = None

pnlData = pd.DataFrame.from_dict(symbolGroupDict, orient="index")
pnlData["symbol"] = pnlData.index
pnlData.columns = ["realized_profit", "symbol"]
pnlData.to_csv("./result_pnl_output.csv", index=None)
